In [1]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [2]:
from static_grader import grader

# DW Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to wrangle tabular data set and aggregate large data sets into meaningful summary statistics. We'll work with the same medical data used in the `pw` miniproject but leverage the power of Pandas to more efficiently represent and act on our data.

## Downloading the data

We first need to download the data we'll be using from Amazon S3:

In [3]:
!mkdir dw-data
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201701scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/201606scripts_sample.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/practices.csv.gz -nc -P ./dw-data/
!wget http://dataincubator-wqu.s3.amazonaws.com/dwdata/chem.csv.gz -nc -P ./dw-data/

mkdir: cannot create directory ‘dw-data’: File exists
File ‘./dw-data/201701scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/201606scripts_sample.csv.gz’ already there; not retrieving.

File ‘./dw-data/practices.csv.gz’ already there; not retrieving.

File ‘./dw-data/chem.csv.gz’ already there; not retrieving.



## Loading the data

Similar to the `PW` miniproject, the first step is to read in the data. The data files are stored as compressed CSV files. You can load the data into a Pandas DataFrame by making use of the `gzip` package to decompress the files and Panda's `read_csv` methods to parse the data into a DataFrame. You may want to check the Pandas documentation for parsing [CSV](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) files for reference.

For a description of the data set please, refer to the [PW miniproject](./pw.ipynb). **Note that all questions make use of the 2017 data only, except for Question 5 which makes use of both the 2017 and 2016 data.**

In [4]:
import pandas as pd
import numpy as np
import gzip

In [5]:
# load the 2017 data
with gzip.open('./dw-data/201701scripts_sample.csv.gz', 'rb') as s:
    scripts = pd.read_csv(s)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6


In [6]:
col_names=[ 'code', 'name', 'addr_1', 'addr_2', 'borough', 'village', 'post_code']
with gzip.open('./dw-data/practices.csv.gz', 'rb') as p:
    practices = pd.read_csv(p, names=col_names)
    
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [7]:
with gzip.open('./dw-data/chem.csv.gz', 'rb') as c:
    chem = pd.read_csv(c)
chem.head()

,CHEM SUB,NAME
0,0101010A0,Alexitol Sodium
1,0101010B0,Almasilate
2,0101010C0,Aluminium Hydroxide
3,0101010D0,Aluminium Hydroxide With Magnesium
4,0101010E0,Hydrotalcite


Now that we've loaded in the data, let's first replicate our results from the `PW` miniproject. Note that we are now working with a larger data set so the answers will be different than in the `PW` miniproject even if the analysis is the same.

## Question 1: summary_statistics

In the `PW` miniproject we first calculated the total, mean, standard deviation, and quartile statistics of the `'items'`, `'quantity'`', `'nic'`, and `'act_cost'` fields. To do this we had to write some functions to calculate the statistics and apply the functions to our data structure. The DataFrame has a `describe` method that will calculate most (not all) of these things for us.

Submit the summary statistics to the grader as a list of tuples: [('act_cost', (total, mean, std, q25, median, q75)), ...]

In [52]:
# summary_stats = [('items', (0,) * 6), ('quantity', (0,) * 6), ('nic', (0,) * 6), ('act_cost', (0,) * 6)]
summary = scripts.describe().transpose()
summary

,count,mean,std,min,25%,50%,75%,max
items,973193.0,9.133136,29.204198,1.00,1.00,2.00,6.00,2384.00
nic,973193.0,73.058915,188.070257,0.00,7.80,22.64,65.00,16320.00
act_cost,973193.0,67.986613,174.401703,0.04,7.33,21.22,60.67,15108.32
quantity,973193.0,741.329835,3665.426958,0.00,28.00,100.00,350.00,577720.00
opioids,973193.0,0.035803,0.185798,0.00,0.00,0.00,0.00,1.00


In [53]:
summary['total'] = summary['mean'] * summary['count'].astype('int')
summary



,count,mean,std,min,25%,50%,75%,max,total
items,973193.0,9.133136,29.204198,1.00,1.00,2.00,6.00,2384.00,8.888304e+06
nic,973193.0,73.058915,188.070257,0.00,7.80,22.64,65.00,16320.00,7.110042e+07
act_cost,973193.0,67.986613,174.401703,0.04,7.33,21.22,60.67,15108.32,6.616410e+07
quantity,973193.0,741.329835,3665.426958,0.00,28.00,100.00,350.00,577720.00,7.214570e+08
opioids,973193.0,0.035803,0.185798,0.00,0.00,0.00,0.00,1.00,3.484300e+04


In [56]:
summary = summary[['total', 'mean', 'std', '25%', '50%', '75%']]
summary_stats = [(row[0], row[1:]) for row in summary.itertuples()]
summary_stats

[('items', (8888304.0, 9.133135976111625, 29.204198282803603, 1.0, 2.0, 6.0)),
 ('nic',
  (71100424.84000002, 73.05891517920908, 188.070256906825, 7.8, 22.64, 65.0)),
 ('act_cost',
  (66164096.11999998,
   67.98661326170655,
   174.40170332301963,
   7.33,
   21.22,
   60.67)),
 ('quantity',
  (721457006.0, 741.3298348837282, 3665.426958467915, 28.0, 100.0, 350.0)),
 ('opioids',
  (34843.0, 0.03580276471367961, 0.18579817605238425, 0.0, 0.0, 0.0))]

In [ ]:
grader.score.dw__summary_statistics(summary_stats)

## Question 2: most_common_item

We can also easily compute summary statistics on groups within the data. In the `pw` miniproject we had to explicitly construct the groups based on the values of a particular field. Pandas will handle that for us via the `groupby` method. This process is [detailed in the Pandas documentation](https://pandas.pydata.org/pandas-docs/stable/groupby.html).

Use `groupby` to calculate the total number of items dispensed for each `'bnf_name'`. Find the item with the highest total and return the result as `[(bnf_name, total)]`.

In [45]:
totals = scripts.groupby('bnf_name')['items'].sum()
totals.head()
totals.max()

218583

In [46]:
totals.sort_values()

bnf_name
ReadyWrap Gauntlet Lymph Gmt                     1
Quetiapine_Oral Soln 12.5mg/5ml                  1
Juzo Soft Rib Style Class 1 B/Knee Close         1
Quest_Lactase Tab                                1
Quest_Cell Life Antiox Tab                       1
                                             ...  
Amlodipine_Tab 5mg                          128245
Simvastatin_Tab 40mg                        132941
Aspirin Disper_Tab 75mg                     148591
Paracet_Tab 500mg                           151669
Omeprazole_Cap E/C 20mg                     218583
Name: items, Length: 13471, dtype: int64

In [ ]:
most_common_item = [(totals.idxmax(), totals[totals.idxmax()])]


In [ ]:
grader.score.dw__most_common_item(most_common_item)

## Question 3: items_by_region

Now let's find the most common item by post code. The post code information is in the `practices` DataFrame, and we'll need to `merge` it into the `scripts` DataFrame. Pandas provides [extensive documentation](https://pandas.pydata.org/pandas-docs/stable/merging.html) with diagrammed examples on different methods and approaches for joining data. The `merge` method is only one of many possible options.

Return your results as a list of tuples `(post code, item name, amount dispensed as % of total)`. Sort your results ascending alphabetically by post code and take only results from the first 100 post codes.

**NOTE:** Some practices have multiple postal codes associated with them. Use the alphabetically first postal code. Note some postal codes may have multiple `'bnf_name'` with the same prescription rate for the maximum. In this case, take the alphabetically first `'bnf_name'` (as in the PW miniproject).

In [33]:
practices.head()

,code,name,addr_1,addr_2,borough,village,post_code
0,A81001,THE DENSHAM SURGERY,THE HEALTH CENTRE,LAWSON STREET,STOCKTON ON TEES,CLEVELAND,TS18 1HU
1,A81002,QUEENS PARK MEDICAL CENTRE,QUEENS PARK MEDICAL CTR,FARRER STREET,STOCKTON ON TEES,CLEVELAND,TS18 2AW
2,A81003,VICTORIA MEDICAL PRACTICE,THE HEALTH CENTRE,VICTORIA ROAD,HARTLEPOOL,CLEVELAND,TS26 8DB
3,A81004,WOODLANDS ROAD SURGERY,6 WOODLANDS ROAD,NaN,MIDDLESBROUGH,CLEVELAND,TS1 3BE
4,A81005,SPRINGWOOD SURGERY,SPRINGWOOD SURGERY,RECTORY LANE,GUISBOROUGH,NaN,TS14 7DJ


In [34]:

scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,opioids
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,0
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,0
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,0
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,0
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,0


In [35]:
unique_practices = (practices.sort_values('post_code').groupby('code').first()
    .reset_index())[['code', 'post_code']]
unique_practices.head()

,code,post_code
0,A81001,TS18 1HU
1,A81002,TS18 2AW
2,A81003,TS25 1QU
3,A81004,TS1 3BE
4,A81005,TS14 7DJ


In [36]:
joined = scripts.merge(unique_practices, left_on='practice', right_on='code', how='left')
joined.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,opioids,code,post_code
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,0,N85639,CH44 5UF
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,0,N85639,CH44 5UF
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,0,N85639,CH44 5UF
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,0,N85639,CH44 5UF
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,0,N85639,CH44 5UF


In [37]:
# total item for each bnf_name in each postcode
post_item_totals = (joined.groupby(['post_code', 'bnf_name'])['items']
    .sum()
    .reset_index()
    .sort_values(['post_code', 'items'], ascending=False))

post_item_totals.head()



,post_code,bnf_name,items
576696,YO16 4LZ,Aspirin Disper_Tab 75mg,1194
576703,YO16 4LZ,Atorvastatin_Tab 20mg,1122
578363,YO16 4LZ,Omeprazole_Cap E/C 20mg,1119
577959,YO16 4LZ,Lansoprazole_Cap 30mg (E/C Gran),1044
578443,YO16 4LZ,Paracet_Tab 500mg,1012


In [38]:
post_code_totals = post_item_totals.groupby('post_code')['items'].sum().head()
post_code_totals

post_code
B11 4BW    22731
B12 9LP    17073
B18 7AL    20508
B21 9RY    31027
B23 6DJ    28011
Name: items, dtype: int64

In [39]:
max_items = post_item_totals.groupby('post_code').first()
max_items['post_totals'] = post_code_totals
max_items['proportion'] = max_items['items'] / max_items['post_totals']
max_items.head()

,bnf_name,items,post_totals,proportion
post_code,,,,
B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,706,22731.0,0.031059
B12 9LP,Paracet_Tab 500mg,425,17073.0,0.024893
B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,556,20508.0,0.027111
B21 9RY,Metformin HCl_Tab 500mg,1033,31027.0,0.033294
B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),599,28011.0,0.021384


In [40]:
max_items.drop(['items', 'post_totals'], axis=1, inplace=True)
max_items.head()


,bnf_name,proportion
post_code,,
B11 4BW,Salbutamol_Inha 100mcg (200 D) CFF,0.031059
B12 9LP,Paracet_Tab 500mg,0.024893
B18 7AL,Salbutamol_Inha 100mcg (200 D) CFF,0.027111
B21 9RY,Metformin HCl_Tab 500mg,0.033294
B23 6DJ,Lansoprazole_Cap 30mg (E/C Gran),0.021384


In [44]:
items_by_region = list(max_items.itertuples())[:100]


In [42]:
grader.score.dw__items_by_region(items_by_region)

Your score:  1.0


## Question 4: script_anomalies

Drug abuse is a source of human and monetary costs in health care. A first step in identifying practitioners that enable drug abuse is to look for practices where commonly abused drugs are prescribed unusually often. Let's try to find practices that prescribe an unusually high amount of opioids. The opioids we'll look for are given in the list below.

In [8]:
opioids = ['morphine', 'oxycodone', 'methadone', 'fentanyl', 'pethidine', 'buprenorphine', 'propoxyphene', 'codeine']

In [13]:
mask = chem['NAME'].str.contains('|'.join(opioids), case=False)
opioid_codes = chem[mask]['CHEM SUB']
opioid_codes.head()

88     0104020D0
91     0104020N0
691    0309010C0
693    0309010N0
695    0309010S0
Name: CHEM SUB, dtype: object

In [18]:
scripts['opioids'] = scripts['bnf_code'].isin(opioid_codes).astype(int)
scripts.head()

,practice,bnf_code,bnf_name,items,nic,act_cost,quantity,opioids
0,N85639,0106020C0,Bisacodyl_Tab E/C 5mg,1,0.39,0.47,12,0
1,N85639,0106040M0,Movicol Plain_Paed Pdr Sach 6.9g,1,4.38,4.07,30,0
2,N85639,0301011R0,Salbutamol_Inha 100mcg (200 D) CFF,1,1.50,1.40,1,0
3,N85639,0304010G0,Chlorphenamine Mal_Oral Soln 2mg/5ml,1,2.62,2.44,150,0
4,N85639,0401020K0,Diazepam_Tab 2mg,1,0.16,0.26,6,0


These are generic names for drugs, not brand names. Generic drug names can be found using the `'bnf_code'` field in `scripts` along with the `chem` table.. Use the list of opioids provided above along with these fields to make a new field in the `scripts` data that flags whether the row corresponds with a opioid prescription.

Now for each practice calculate the proportion of its prescriptions containing opioids.

**Hint:** Consider the following list: `[0, 1, 1, 0, 0, 0]`. What proportion of the entries are 1s? What is the mean value?

In [20]:
scripts.groupby('practice')['opioids'].mean().head()

practice
A81005    0.033179
A81007    0.043329
A81011    0.046556
A81012    0.042793
A81017    0.038140
Name: opioids, dtype: float64

In [21]:
opioids_per_practice = scripts.groupby('practice')['opioids'].mean()
opioids_per_practice.head()

practice
A81005    0.033179
A81007    0.043329
A81011    0.046556
A81012    0.042793
A81017    0.038140
Name: opioids, dtype: float64

How do these proportions compare to the overall opioid prescription rate? Subtract off the proportion of all prescriptions that are opioids from each practice's proportion.

In [22]:
relative_opioids_per_practice = (opioids_per_practice - scripts['opioids'].mean())
relative_opioids_per_practice.head()

practice
A81005   -0.002624
A81007    0.007526
A81011    0.010753
A81012    0.006990
A81017    0.002337
Name: opioids, dtype: float64

In [25]:
np.sqrt(scripts['opioids'].var() / scripts['practice'].value_counts()).head()



N83028    0.003484
L83100    0.003513
D81043    0.003747
B81008    0.003796
B81026    0.003835
Name: practice, dtype: float64

In [26]:
standard_error_practice = np.sqrt(scripts['opioids'].var() / scripts['practice'].value_counts())


opioid_scores = relative_opioids_per_practice / standard_error_practice


Now that we know the difference between each practice's opioid prescription rate and the overall rate, we can identify which practices prescribe opioids at above average or below average rates. However, are the differences from the overall rate important or just random deviations? In other words, are the differences from the overall rate big or small?

To answer this question we have to quantify the difference we would typically expect between a given practice's opioid prescription rate and the overall rate. This quantity is called the **standard error**, and is related to the **standard deviation**, $\sigma$. The standard error in this case is

$$ \frac{\sigma}{\sqrt{n}} $$

where $n$ is the number of prescriptions each practice made. Calculate the standard error for each practice. Then divide `relative_opioids_per_practice` by the standard errors. We'll call the final result `opioid_scores`.

In [27]:
unique_practices = ( practices.sort_values(['code', 'name'])
                    .drop_duplicates(subset=['code'], keep='first'))

unique_practices = unique_practices[['code', 'name']]
unique_practices.head()

,code,name
0,A81001,THE DENSHAM SURGERY
1,A81002,QUEENS PARK MEDICAL CENTRE
2,A81003,VICTORIA MEDICAL PRACTICE
11915,A81004,BLUEBELL MEDICAL CENTRE
4,A81005,SPRINGWOOD SURGERY


In [28]:
results = unique_practices.merge(opioid_scores.rename('score'), 
                               left_on = 'code',
                                right_index = True)

results = results.merge(scripts['practice'].value_counts().rename('count'),
                       left_on = 'code',
                        right_index = True)

results.sort_values(by='score', ascending=False, inplace=True)
results.drop('code', axis=1, inplace=True)
results.head()

,name,score,count
7954,NATIONAL ENHANCED SERVICE,11.695818,7
8456,OUTREACH SERVICE NH / RH,7.339043,2
8783,BRISDOC HEALTHCARE SERVICES OOH,6.150582,60
7454,H&R P C SPECIAL SCHEME,5.123032,36
9718,HMR BARDOC OOH,4.958866,321


The quantity we have calculated in `opioid_scores` is called a **z-score**:

$$ \frac{\bar{X} - \mu}{\sqrt{\sigma^2/n}} $$

Here $\bar{X}$ corresponds with the proportion for each practice, $\mu$ corresponds with the proportion across all practices, $\sigma^2$ corresponds with the variance of the proportion across all practices, and $n$ is the number of prescriptions made by each practice. Notice $\bar{X}$ and $n$ will be different for each practice, while $\mu$ and $\sigma$ are determined across all prescriptions, and so are the same for every z-score. The z-score is a useful statistical tool used for hypothesis testing, finding outliers, and comparing data about different types of objects or events.

Now that we've calculated this statistic, take the 100 practices with the largest z-score. Return your result as a list of tuples in the form `(practice_name, z-score, number_of_scripts)`. Sort your tuples by z-score in descending order. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [29]:
anomalies = list(results.itertuples(index=False, name=None))[:100]
anomalies[:2]

[('NATIONAL ENHANCED SERVICE', 11.695817862936027, 7),
 ('OUTREACH SERVICE NH / RH', 7.339043019238823, 2)]

In [30]:
results.head()

,name,score,count
7954,NATIONAL ENHANCED SERVICE,11.695818,7
8456,OUTREACH SERVICE NH / RH,7.339043,2
8783,BRISDOC HEALTHCARE SERVICES OOH,6.150582,60
7454,H&R P C SPECIAL SCHEME,5.123032,36
9718,HMR BARDOC OOH,4.958866,321


In [31]:
grader.score.dw__script_anomalies(anomalies)

Your score:  1.0


## Question 5: script_growth

Another way to identify anomalies is by comparing current data to historical data. In the case of identifying sites of drug abuse, we might compare a practice's current rate of opioid prescription to their rate 5 or 10 years ago. Unless the nature of the practice has changed, the profile of drugs they prescribe should be relatively stable. We might also want to identify trends through time for business reasons, identifying drugs that are gaining market share. That's what we'll do in this question.

We'll load in beneficiary data from 6 months earlier, June 2016, and calculate the percent growth in prescription rate from June 2016 to January 2017 for each `bnf_name`. We'll return the 50 items with largest growth and the 50 items with the largest shrinkage (i.e. negative percent growth) as a list of tuples sorted by growth rate in descending order in the format `(script_name, growth_rate, raw_2016_count)`. You'll notice that many of the 50 fastest growing items have low counts of prescriptions in 2016. Filter out any items that were prescribed less than 50 times.

In [ ]:
scripts16 = pd.read_csv('./dw-data/201606scripts_sample.csv.gz')
scripts16.head()

In [ ]:
pct_growth = (scripts.bnf_name.value_counts() - scripts16.bnf_name.value_counts()) / scripts16.bnf_name.value_counts()
pct_growth.dropna(inplace=True)

pct_growth.sort_values(ascending=False).head()

In [ ]:
scripts16.bnf_name.value_counts().head()

In [ ]:
script_growth_df = pd.concat([pct_growth, scripts16.bnf_name.value_counts()],
                             axis=1, sort=True)
script_growth_df.columns = ['pct_growth', 'count16']


In [ ]:
script_growth_df.dropna(inplace=True)


In [ ]:
script_growth_df.head()

In [ ]:
script_growth_df.shape

In [ ]:
mask = script_growth_df['count16'] >= 50
script_growth_df = script_growth_df[mask]
script_growth_df.shape

In [ ]:
script_growth_df.sort_values('pct_growth', ascending=False, inplace=True)



In [ ]:
script_growth_df = pd.concat([script_growth_df.head(50), 
                              script_growth_df.tail(50)])


In [ ]:
script_growth_df.head()

In [ ]:
script_growth = list(script_growth_df.itertuples(name=None))
script_growth[:5]

In [ ]:
grader.score.dw__script_growth(script_growth)

## Question 6: rare_scripts

Does a practice's prescription costs originate from routine care or from reliance on rarely prescribed treatments? Commonplace treatments can carry lower costs than rare treatments because of efficiencies in large-scale production. While some specialist practices can't help but avoid prescribing rare medicines because there are no alternatives, some practices may be prescribing a unnecessary amount of brand-name products when generics are available. Let's identify practices whose costs disproportionately originate from rarely prescribed items.

First we have to identify which `'bnf_code'` are rare. To do this, find the probability $p$ of a prescription having a particular `'bnf_code'` if the `'bnf_code'` was randomly chosen from the unique options in the beneficiary data. We will call a `'bnf_code'` rare if it is prescribed at a rate less than $0.1p$.

In [ ]:
#find number of unique prescription
p = 1 / scripts['bnf_code'].nunique()
print(p)

#find rate of prescription (count/total)
rates  = scripts['bnf_code'].value_counts() / len(scripts)
rates.head()


In [ ]:
mask = rates < .1 * p
mask.head()

type(mask)

rare_codes = rates[mask].index.unique()
rare_codes



In [ ]:
scripts['rare'] = scripts['bnf_code'].isin(rare_codes)
scripts['rare'].sum()



In [ ]:


# p = ...
# rates = ...
# rare_codes = ...
# scripts['rare'] = ...

Now for each practice, calculate the proportion of costs that originate from prescription of rare treatments (i.e. rare `'bnf_code'`). Use the `'act_cost'` field for this calculation.

In [ ]:
rare_cost_prop = (scripts[scripts['rare']].groupby('practice')['act_cost'].sum()  / scripts.groupby('practice')['act_cost'].sum()).fillna(0)

rare_cost_prop.head()


Now we will calculate a z-score for each practice based on this proportion.
First take the difference of `rare_cost_prop` and the proportion of costs originating from rare treatments across all practices.

In [ ]:
relative_rare_cost_prop = (rare_cost_prop 
    - scripts[scripts['rare']]['act_cost'].sum()
    /scripts['act_cost'].sum())

relative_rare_cost_prop.head()

Now we will estimate the standard errors (i.e. the denominator of the z-score) by simply taking the standard deviation of this difference.

In [ ]:
standard_errors = relative_rare_cost_prop.std()



In [ ]:
unique_practices = (practices.sort_values('name')
                   .groupby('code')
                   .first().
                   reset_index())[['code', 'name']]
unique_practices.set_index('code', inplace=True)
unique_practices.head()



Finally compute the z-scores. Return the practices with the top 100 z-scores in the form `(post_code, practice_name, z-score)`. Note that some practice codes will correspond with multiple names. In this case, use the first match when sorting names alphabetically.

In [ ]:

rare_scores = (relative_rare_cost_prop / standard_errors)

In [ ]:
rare_scores = pd.concat([unique_practices, rare_scores], axis = 1)\
.sort_values('act_cost', ascending=False).head(100)

#turn rare scores to lis of tuples
rare_scripts = list(rare_scores.itertuples(name=None))
rare_scripts[:3]

In [ ]:
# rare_scripts = [("Y03472", "CONSULTANT DIABETES TEAM", 16.2626871247)] * 100

In [ ]:
grader.score.dw__rare_scripts(rare_scripts)

*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*